<a href="https://colab.research.google.com/github/janghyukjin/2019_cau_oss_hackathon/blob/master/hackathon_team_CVML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
 *  단, 사용할 데이터셋에 따라 is_mnist만 수정
*   모든 구현은 [2. 데이터 전처리]와 [3. 모델 생성]에서만 진행
 *  데이터 전처리 후 트레이닝, 데이터 셋은 x_train_after, x_test_after 변수명을 유지해주세요.
 *  데이터셋이 달라져도 같은 모델 구조를 유지하여야함.
*   [4. 모델 저장]과 [5. 모델 로드 및 평가]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *  트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
 *  team_name을 제외한 다른 부분은 수정하지 말 것
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  모델 구조 파일 (model_structure_teamXX.json)
 *  모델 weight 파일 (model_weight_teamXX.h5)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
* 제출 기한: **오후 6시**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2019_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  두 개의 데이터셋에 대해 다른 모델 구조를 사용한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
#from tensorflow import keras
import keras
from keras.utils import np_utils


import numpy as np
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

import os
import cv2


is_mnist = False;

# 데이터셋 로드
# x_train, y_train: 트레이닝 데이터 및 레이블
# x_test, y_test: 테스트 데이터 및 레이블
if is_mnist:
  data_type = 'mnist'
  (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data() # fashion MNIST 데이터셋인 경우,
else:
  data_type = 'cifar10'
  (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data() # cifar10 데이터셋인 경우,


# 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 총 클래스 개수
num_classes = y_test.shape[1]

# 인풋 데이터 타입
input_shape = x_test.shape[1:]

# **2. 데이터 전처리**


In [0]:
# 데이터 전처리 (예: normalization)

MODEL_SAVE_FOLDER_PATH = './mode55/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
  os.mkdir(MODEL_SAVE_FOLDER_PATH)
model_path = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_acc:.4f}.h5'

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc', save_best_only=True, save_weights_only=True,
                                verbose=1)




    
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0

#x_train_after = cv2.normalize(x_train, x_train, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
#x_test_after = cv2.normalize(x_test, x_test, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)




if is_mnist:
    x_train_after = x_train_after.reshape(x_train_after.shape[0],28,28,1)
    x_test_after = x_test_after.reshape(x_test_after.shape[0],28,28,1)
    bgs = cv2.createBackgroundSubtractorMOG2()
    for a in x_train_after:
       a = cv2.pyrDown(a)
       a = bgs.apply(a)
    for a in x_test_after:
       a = cv2.pyrDown(a)
       a = bgs.apply(a)
      
    input_shape = x_test_after.shape[1:]
else:
  bgs = cv2.createBackgroundSubtractorMOG2()
  for a in x_train_after:
     a = cv2.pyrDown(a)
     a = bgs.apply(a)
  for a in x_test_after:
     a = cv2.pyrDown(a)
     a = bgs.apply(a)



# **3. 모델 생성**



In [0]:




# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

#model.add(Conv2D(16, kernel_size=(5,5),  activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(32, kernel_size=(5,5),  activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,kernel_size=(3,3),input_shape=input_shape,  activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),input_shape=input_shape,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
#model.add(keras.layers.Flatten())

model.add(Conv2D(128,kernel_size=(3,3),input_shape=input_shape,  activation='relu'))
model.add(Conv2D(256,kernel_size=(3,3),input_shape=input_shape,  activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

'''
model.add(Conv2D(512,kernel_size=(1,1),input_shape=input_shape,  activation='relu'))
model.add(Conv2D(512,kernel_size=(3,3),input_shape=input_shape,  activation='relu'))
model.add(Conv2D(1024,kernel_size=(1,1),input_shape=input_shape,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))




model.add(Conv2D(128,kernel_size=(3,3),input_shape=input_shape,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,kernel_size=(3,3),input_shape=input_shape,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5)) # 92..

model.add(Conv2D(32,kernel_size=(3,3), input_shape=input_shape,  activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
'''

#model.add(Conv2D(filters=12, kernel_size=(5,5),  activation='sigmoid'))
#model.add(MaxPooling2D(strides=(2,2),pool_size=(2, 2)))


#model.add(Flatten())

# Flatten layer: 28 x 28 x 1 image를 784개의 1D vector input으로 변환
model.add(keras.layers.Flatten())

# 1st hidden layer: fully-connected layer
# (# of inputs = 784, # of outputs = 512, actication fuction = relu)
model.add(keras.layers.Dense(392, activation=tf.nn.relu))

# 2nd hidden layer: fully-connected layer 
# (# of inputs = 512, # of outputs = 256, actication fuction = relu)
model.add(keras.layers.Dense(196, activation=tf.nn.relu))

# 3rd hidden layer: fully-connected layer 
# (# of inputs = 256, # of outputs = 64, actication fuction = relu)
model.add(keras.layers.Dense(98, activation=tf.nn.relu))

# Output layer: fully-connected layer 
# (# of inputs = 64, # of outputs = 10, actication fuction = softmax)
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set


model.fit(x_train_after, y_train, batch_size = 128, epochs = 75, shuffle=True, validation_data=[x_test_after, y_test], callbacks=[cb_checkpoint])

Train on 50000 samples, validate on 10000 samples
Epoch 1/75
50000/50000 [==============================] - 16s 324us/step - loss: 1.6449 - acc: 0.3903 - val_loss: 1.2830 - val_acc: 0.5375

Epoch 00001: val_acc improved from -inf to 0.53750, saving model to ./mode55/01-0.5375.h5
Epoch 2/75
50000/50000 [==============================] - 10s 194us/step - loss: 1.1776 - acc: 0.5759 - val_loss: 1.0238 - val_acc: 0.6368

Epoch 00002: val_acc improved from 0.53750 to 0.63680, saving model to ./mode55/02-0.6368.h5
Epoch 3/75
50000/50000 [==============================] - 10s 194us/step - loss: 0.9604 - acc: 0.6606 - val_loss: 0.8398 - val_acc: 0.7090

Epoch 00003: val_acc improved from 0.63680 to 0.70900, saving model to ./mode55/03-0.7090.h5
Epoch 4/75
50000/50000 [==============================] - 10s 193us/step - loss: 0.8261 - acc: 0.7082 - val_loss: 0.7815 - val_acc: 0.7251

Epoch 00004: val_acc improved from 0.70900 to 0.72510, saving model to ./mode55/04-0.7251.h5
Epoch 5/75
50000/5000

# **4. 모델 저장**

In [0]:
save_path = '/content/'
team_name = 'teamCVML'

# 모델의 weight 값만 저장합니다.
model.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')

# 모델의 구조만을 저장합니다.
model_json = model.to_json()
with open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_' + data_type + '_' + team_name + '.h5')

# **5. 모델 로드 및 평가**

In [0]:
save_path = '/content/'
team_name = 'teamCVML'

model = keras.models.load_model(save_path + 'model_entire_' + data_type + '_' + team_name + '.h5')
model.evaluate(x_test_after, y_test)

10000/10000 [==============================] - 4s 388us/step


[0.9082480331659317, 0.8036]